# CGmap YOLOv8 - Complete Tutorial

This notebook demonstrates the complete workflow for crop gap detection using YOLOv8.

## Contents
1. Setup and Installation
2. Data Preparation
3. Model Training
4. Evaluation
5. Inference
6. Visualization
7. Model Export

## 1. Setup and Installation

In [ ]:
# Install required packages (uncomment if needed)
# !pip install ultralytics

# Import libraries
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Import project utilities
from utils.data_preprocessing import create_sample_data, validate_dataset
from ultralytics import YOLO

print("✓ Setup complete!")

## 2. Data Preparation

In [ ]:
# Create sample dataset for demonstration
data_dir = project_root / 'data' / 'processed'

print("Creating sample dataset...")
create_sample_data(data_dir, num_samples=50)

print("\nValidating dataset...")
validate_dataset(data_dir, num_classes=2)

print("\n✓ Data preparation complete!")

In [ ]:
# Visualize sample images
import cv2

train_images = sorted((data_dir / 'train' / 'images').glob('*.jpg'))[:4]

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.ravel()

for idx, img_path in enumerate(train_images):
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(img)
    axes[idx].set_title(f"Sample {idx+1}: {img_path.name}")
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 3. Model Training

We'll train a YOLOv8 nano model for quick demonstration. For better results, use yolov8s/m/l with more epochs.

In [ ]:
# Initialize model
model = YOLO('yolov8n.pt')

print("Model initialized successfully!")
print(f"Model type: YOLOv8 Nano")

In [ ]:
# Train the model
# Note: This is a minimal example with 3 epochs. For real training, use 100+ epochs

results = model.train(
    data=str(project_root / 'configs' / 'data.yaml'),
    epochs=3,  # Use 100+ for real training
    imgsz=640,
    batch=8,
    device='cpu',  # Change to 0 for GPU
    project='results',
    name='notebook_train',
    exist_ok=True,
    verbose=True
)

print("\n✓ Training complete!")

In [ ]:
# Display training results
results_dir = Path('results/notebook_train')

# Show results plot
if (results_dir / 'results.png').exists():
    display(Image(filename=str(results_dir / 'results.png')))
else:
    print("Training results plot not available yet")

## 4. Model Evaluation

In [ ]:
# Load best model
best_model_path = results_dir / 'weights' / 'best.pt'

if best_model_path.exists():
    model = YOLO(str(best_model_path))
    print(f"Loaded model: {best_model_path}")
else:
    print("Best model not found, using last trained model")

In [ ]:
# Run validation
metrics = model.val()

# Print metrics
print("\nValidation Metrics:")
print(f"  mAP@50: {metrics.box.map50:.4f}")
print(f"  mAP@50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.p.mean():.4f}")
print(f"  Recall: {metrics.box.r.mean():.4f}")

In [ ]:
# Display confusion matrix
confusion_matrix_path = results_dir / 'confusion_matrix.png'

if confusion_matrix_path.exists():
    display(Image(filename=str(confusion_matrix_path)))
else:
    print("Confusion matrix not available")

## 5. Inference

Run the model on new images to detect crop gaps.

In [ ]:
# Run inference on validation images
val_images_dir = data_dir / 'val' / 'images'

results = model.predict(
    source=str(val_images_dir),
    conf=0.25,
    save=True,
    project='results',
    name='notebook_predict',
    exist_ok=True
)

print(f"\nProcessed {len(results)} images")
print(f"Results saved to: results/notebook_predict")

In [ ]:
# Display prediction results
predict_dir = Path('results/notebook_predict')
predicted_images = sorted(predict_dir.glob('*.jpg'))[:4]

if predicted_images:
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))
    axes = axes.ravel()
    
    for idx, img_path in enumerate(predicted_images):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"Prediction {idx+1}")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No prediction images found")

In [ ]:
# Analyze predictions
total_detections = sum(len(r.boxes) for r in results)
avg_confidence = np.mean([box.conf.cpu().numpy() for r in results for box in r.boxes]) if results else 0

print(f"\nPrediction Statistics:")
print(f"  Total images: {len(results)}")
print(f"  Total detections: {total_detections}")
print(f"  Avg detections per image: {total_detections / len(results) if results else 0:.2f}")
print(f"  Average confidence: {avg_confidence:.3f}")

## 6. Visualization

Visualize training curves and performance metrics.

In [ ]:
# Plot training curves from CSV
import pandas as pd

results_csv = results_dir / 'results.csv'

if results_csv.exists():
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss curves
    if 'train/box_loss' in df.columns:
        axes[0].plot(df['epoch'], df['train/box_loss'], label='Box Loss', marker='o')
        axes[0].plot(df['epoch'], df['train/cls_loss'], label='Class Loss', marker='s')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Training Losses')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
    
    # mAP curves
    if 'metrics/mAP50(B)' in df.columns:
        axes[1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50', marker='o', linewidth=2)
        if 'metrics/mAP50-95(B)' in df.columns:
            axes[1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@50-95', marker='s', linewidth=2)
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('mAP')
        axes[1].set_title('Mean Average Precision')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Results CSV not found")

## 7. Model Export

Export the trained model to different formats for deployment.

In [ ]:
# Export to ONNX format
try:
    onnx_path = model.export(format='onnx', imgsz=640)
    print(f"✓ Model exported to: {onnx_path}")
except Exception as e:
    print(f"Export failed: {e}")

## Summary

In this notebook, we:
1. Set up the environment and loaded dependencies
2. Created and validated a sample dataset
3. Trained a YOLOv8 model for crop gap detection
4. Evaluated model performance
5. Ran inference on new images
6. Visualized results and metrics
7. Exported the model for deployment

## Next Steps

- Train with your own UAV imagery
- Experiment with different model sizes (yolov8s, yolov8m, yolov8l)
- Adjust hyperparameters for better performance
- Deploy the model on edge devices
- Integrate with geospatial workflows

For more information, see:
- [README.md](README.md)
- [QUICKSTART.md](QUICKSTART.md)
- [DOCUMENTATION.md](DOCUMENTATION.md)